### Train model in notebook (inefficient, for testing and debug only)

In [1]:
import os

In [2]:
#os.environ['NCCL_DEBUG'] = 'INFO'
#os.environ['NCCL_DEBUG_SUBSYS'] = 'ALL'
#os.environ['TORCH_DISTRIBUTED_DEBUG'] = 'INFO'
os.environ['NCCL_IB_DISABLE'] = '1'
os.environ['NCCL_SOCKET_IFNAME'] = 'enp2s0f1np1'
os.environ['NCCL_P2P_DISABLE'] = '1'
os.environ['NCCL_BLOCKING_WAIT'] = '1'

In [2]:
import sys
from os.path import join
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import optim, nn
import torch.multiprocessing as mp
from torch.distributed import init_process_group, destroy_process_group
from torch.utils.data.distributed import DistributedSampler
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import DataLoader
from astropy.io import fits
import pyxis.torch as pxt

from networks import *
from train import *
import config

train_dir = '/data/wxs0703/kl-nn/databases/train_database_5m'
test_dir = '/data/wxs0703/kl-nn/databases/test_database_5m'
fig_dir = '/data/wxs0703/kl-nn/figures/'
model_dir = '/data/wxs0703/kl-nn/models/'

In [3]:
torch.cuda.device_count()

8

In [4]:
world_size = 4
save_every = 1
nepochs = config.train['epoch_number']
batch_size = config.train['batch_size']
nfeatures = config.train['feature_number']

In [3]:
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "12355"
torch.cuda.set_device(0)
init_process_group(backend='nccl', rank=0, world_size=1)

In [4]:
train_ds = pxt.TorchDataset(train_dir)
valid_ds = pxt.TorchDataset(test_dir)

In [5]:
fid_train = torch.empty((100, 9), dtype=torch.float, device='cuda')

In [6]:
for i in range(100):
    fid_train[i, :3] = train_ds[i]['fid_pars'][:3]
    fid_train[i, 4:] = train_ds[i]['fid_pars'][3:]

In [8]:
print(fid_train)

tensor([[-0.9312, -0.2813,  0.8558,  0.0000,  0.2541,  0.4595,  0.7840, -0.7285,
          0.1464],
        [-0.3221, -0.8664, -0.0313,  0.0000, -0.2352,  0.6371, -0.3727, -0.0522,
          0.3316],
        [-0.4979,  0.0033, -0.5870,  0.0000, -0.6996,  0.7740,  0.1849,  0.7089,
         -0.3780],
        [-0.0646, -0.5892,  0.9468,  0.0000,  0.4970, -0.1181, -0.6317,  0.4867,
          0.9943],
        [ 0.8765, -0.2022, -0.7942,  0.0000, -0.3125, -0.5467,  0.2677, -0.5216,
          0.9378],
        [-0.6598, -0.5139, -0.1286,  0.0000, -0.7659,  0.1634,  0.1324, -0.9054,
          0.5314],
        [ 0.4621, -0.7045, -0.1914,  0.0000, -0.2399,  0.6241,  0.8872,  0.3866,
         -0.8967],
        [-0.1887,  0.8482, -0.6532,  0.0000, -0.6752, -0.8768,  0.2400,  0.9227,
         -0.9140],
        [ 0.0796, -0.4447, -0.4961,  0.0000,  0.2443,  0.1132, -0.3413,  0.2737,
         -0.7803],
        [-0.7348, -0.5069, -0.1140,  0.0000,  0.8401,  0.3965, -0.8905,  0.3151,
          0.8866],


In [10]:
fid_train[:, 3] = torch.cos(np.pi*fid_train[:, 2])
fid_train[:, 2] = torch.sin(np.pi*fid_train[:, 2])

In [11]:
print(fid_train)

tensor([[-0.9312, -0.2813,  0.4377, -0.8991,  0.2541,  0.4595,  0.7840, -0.7285,
          0.1464],
        [-0.3221, -0.8664, -0.0980,  0.9952, -0.2352,  0.6371, -0.3727, -0.0522,
          0.3316],
        [-0.4979,  0.0033, -0.9629, -0.2698, -0.6996,  0.7740,  0.1849,  0.7089,
         -0.3780],
        [-0.0646, -0.5892,  0.1663, -0.9861,  0.4970, -0.1181, -0.6317,  0.4867,
          0.9943],
        [ 0.8765, -0.2022, -0.6025, -0.7981, -0.3125, -0.5467,  0.2677, -0.5216,
          0.9378],
        [-0.6598, -0.5139, -0.3931,  0.9195, -0.7659,  0.1634,  0.1324, -0.9054,
          0.5314],
        [ 0.4621, -0.7045, -0.5657,  0.8246, -0.2399,  0.6241,  0.8872,  0.3866,
         -0.8967],
        [-0.1887,  0.8482, -0.8863, -0.4631, -0.6752, -0.8768,  0.2400,  0.9227,
         -0.9140],
        [ 0.0796, -0.4447, -0.9999,  0.0124,  0.2443,  0.1132, -0.3413,  0.2737,
         -0.7803],
        [-0.7348, -0.5069, -0.3505,  0.9366,  0.8401,  0.3965, -0.8905,  0.3151,
          0.8866],


In [7]:
train_dl = DataLoader(
        train_ds,
        batch_size=batch_size,
        pin_memory=True,
        sampler=DistributedSampler(train_ds),
    )
valid_dl = DataLoader(
        valid_ds,
        batch_size=batch_size,
        pin_memory=True,
        sampler=DistributedSampler(valid_ds),
    )

In [8]:
model = ForkCNN(batch_size)
model.to(0)
model = DDP(model, device_ids=[0])

In [9]:
optimizer = optim.SGD(model.parameters(), 
                      lr=config.train['initial_learning_rate'],
                      momentum=config.train['momentum'])

In [10]:
trainer = CNNTrainer(model, nfeatures, train_dl, valid_dl, optimizer, 0, save_every, batch_size)

In [11]:
trainer.train(50)

Batch 0 complete
Batch 100 complete
Batch 200 complete
Batch 300 complete
Batch 400 complete


KeyboardInterrupt: 

In [12]:
destroy_process_group()

In [ ]:
mp.spawn(train_nn, args=(world_size, ForkCNN, CNNTrainer), nprocs=world_size)

INFO:Setup:Initializing
[W416 21:36:29.598625861 Utils.hpp:164] Warning: Environment variable NCCL_BLOCKING_WAIT is deprecated; use TORCH_NCCL_BLOCKING_WAIT instead (function operator())
INFO:Setup:[rank: 0] Successfully set up device
[rank0]:[W416 21:36:30.282120524 Utils.hpp:110] Warning: Environment variable NCCL_BLOCKING_WAIT is deprecated; use TORCH_NCCL_BLOCKING_WAIT instead (function operator())
INFO:Setup:[rank: 0] Successfully loaded training objects
INFO:Setup:[rank: 0] Successfully prepared dataloader
INFO:Setup:[rank: 0] Successfully initialized Trainer
INFO:Trainer:Batch 0 complete
